## Занятие «Join, exists, вложенные запросы, group by, having. Индексы и план запроса»

In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
connection = create_connection("C:\\Disc_D\\DS_course_mipt\\ОТ\\Python_DA\\БД_SQL\\users_11_11.db")

Connection to SQLite DB successful


### Создание таблиц

In [4]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [5]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  nationality TEXT
);
"""

In [6]:
execute_query(connection, create_users_table)  

Query executed successfully


In [7]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""

In [8]:
execute_query(connection, create_posts_table)

Query executed successfully


In [9]:
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

execute_query(connection, create_comments_table)  
execute_query(connection, create_likes_table)     

Query executed successfully
Query executed successfully


### Добавление записей

In [10]:
create_users = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users)  

Query executed successfully


In [11]:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

execute_query(connection, create_posts) 

Query executed successfully


In [12]:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, create_comments)
execute_query(connection, create_likes)

Query executed successfully
Query executed successfully


### Извлечение данных из записей

In [13]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [14]:
select_users = "SELECT name, age from users where name like 'J%'"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

('James', 25)
('James', 25)


In [15]:
select_posts = "SELECT * FROM posts"
posts = execute_read_query(connection, select_posts)

for post in posts:
    print(post)

(1, 'Happy', 'I am feeling very happy today', 1)
(2, 'Hot Weather', 'The weather is very hot today', 2)
(3, 'Help', 'I need some help with my work', 2)
(4, 'Great News', 'I am getting married', 1)
(5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(6, 'Party', 'Anyone up for a late-night party today?', 3)
(7, 'Happy', 'I am feeling very happy today', 1)
(8, 'Hot Weather', 'The weather is very hot today', 2)
(9, 'Help', 'I need some help with my work', 2)
(10, 'Great News', 'I am getting married', 1)
(11, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(12, 'Party', 'Anyone up for a late-night party today?', 3)


### Джойны таблиц

In [16]:
sql_inner = '''SELECT name, age, gender, nationality, title, description 
FROM users
INNER JOIN posts
ON users.id = posts.user_id;'''

In [17]:
execute_query(connection, sql_inner)

Query executed successfully


In [18]:
posts_users = execute_read_query(connection, sql_inner)

for i in posts_users:
    print(i)

('James', 25, 'male', 'USA', 'Happy', 'I am feeling very happy today')
('Leila', 32, 'female', 'France', 'Hot Weather', 'The weather is very hot today')
('Leila', 32, 'female', 'France', 'Help', 'I need some help with my work')
('James', 25, 'male', 'USA', 'Great News', 'I am getting married')
('Elizabeth', 21, 'female', 'Canada', 'Interesting Game', 'It was a fantastic game of tennis')
('Brigitte', 35, 'female', 'England', 'Party', 'Anyone up for a late-night party today?')
('James', 25, 'male', 'USA', 'Happy', 'I am feeling very happy today')
('Leila', 32, 'female', 'France', 'Hot Weather', 'The weather is very hot today')
('Leila', 32, 'female', 'France', 'Help', 'I need some help with my work')
('James', 25, 'male', 'USA', 'Great News', 'I am getting married')
('Elizabeth', 21, 'female', 'Canada', 'Interesting Game', 'It was a fantastic game of tennis')
('Brigitte', 35, 'female', 'England', 'Party', 'Anyone up for a late-night party today?')


In [19]:
sql_left = '''SELECT name, age, gender, nationality, title, description 
FROM users
LEFT JOIN posts
ON users.id = posts.user_id;'''

In [20]:
posts_users = execute_read_query(connection, sql_left)

for i in posts_users:
    print(i)

('James', 25, 'male', 'USA', 'Great News', 'I am getting married')
('James', 25, 'male', 'USA', 'Great News', 'I am getting married')
('James', 25, 'male', 'USA', 'Happy', 'I am feeling very happy today')
('James', 25, 'male', 'USA', 'Happy', 'I am feeling very happy today')
('Leila', 32, 'female', 'France', 'Help', 'I need some help with my work')
('Leila', 32, 'female', 'France', 'Help', 'I need some help with my work')
('Leila', 32, 'female', 'France', 'Hot Weather', 'The weather is very hot today')
('Leila', 32, 'female', 'France', 'Hot Weather', 'The weather is very hot today')
('Brigitte', 35, 'female', 'England', 'Party', 'Anyone up for a late-night party today?')
('Brigitte', 35, 'female', 'England', 'Party', 'Anyone up for a late-night party today?')
('Mike', 40, 'male', 'Denmark', None, None)
('Elizabeth', 21, 'female', 'Canada', 'Interesting Game', 'It was a fantastic game of tennis')
('Elizabeth', 21, 'female', 'Canada', 'Interesting Game', 'It was a fantastic game of tenni

### SQL EXISTS

In [21]:
connection = create_connection("C:\\Disc_D\\DS_course_mipt\\ОТ\\Python_DA\\БД_SQL\\users_11_11.db")

Connection to SQLite DB successful


In [22]:
Bookinuse = '''CREATE TABLE  IF NOT EXISTS Bookinuse (
  Author NVARCHAR(50),
  Title NVARCHAR(50),
  Pubyear INT,
  Inv_No INT PRIMARY KEY,
  Customer_ID INT REFERENCES Customer (Customer_ID)
);
'''

Bookinuse_insert = '''INSERT INTO  IF NOT EXISTS Bookinuse (Author, Title, Pubyear, Inv_No, Customer_ID) VALUES
    ('Толстой', 'Война и мир', 2005, 28, 65),
    ('Чехов', 'Вишневый сад', 2000, 20, 31),
    ('Чехов', 'Избранные рассказы', 2011, 19, 120),
    ('Чехов', 'Вишневый сад', 1991, 5, 65),
    ('Ильф и Петров', 'Двенадцать стульев', 1985, 3, 31),
    ('Маяковский', 'Поэмы', 1983, 2, 120),
    ('Пастернак', 'Доктор Живаго', 2006, 69, 120),
    ('Толстой', 'Воскресенье', 2006, 77, 47),
    ('Толстой', 'Анна Каренина', 1989, 7, 205),
    ('Пушкин', 'Капитанская дочка', 2004, 25, 47),
    ('Гоголь', 'Пьесы', 2007, 81, 47),
    ('Чехов', 'Избранные рассказы', 1987, 4, 205),
    ('Пушкин', 'Сочинения т.1', 1984, 6, 47),
    ('Пастернак', 'Избранное', 200, 137, 18),
    ('Пушкин', 'Сочинения т.2', 1984, 8, 205),
    (NULL, 'Наука и жизнь 9 2018', 2019, 127, 18),
    ('Чехов', 'Ранние рассказы', 2001, 171, 31);
'''

In [23]:
execute_query(connection, Bookinuse)

Query executed successfully


In [24]:
execute_query(connection, Bookinuse_insert)

The error 'near "NOT": syntax error' occurred


In [25]:
test_Bookinuse = '''SELECT * FROM Bookinuse;
'''

In [26]:
execute_query(connection, test_Bookinuse)

Query executed successfully


In [27]:
test = execute_read_query(connection, test_Bookinuse)

for i in test:
    print(i)

In [28]:
Customers = '''CREATE TABLE  IF NOT EXISTS Customer (
  Customer_ID INT PRIMARY KEY,
  Surname NVARCHAR(50) NOT NULL
);
'''

Customers_insert = '''INSERT INTO Customer (Customer_ID, Surname) VALUES
(18, 'Зотов'),
(31, 'Перов'),
(47, 'Васин'),
(65, 'Тихонов'),
(120, 'Краснов'),
(205, 'Климов');
'''

In [29]:
execute_query(connection, Customers)

Query executed successfully


In [30]:
execute_query(connection, Customers_insert)

The error 'UNIQUE constraint failed: Customer.Customer_ID' occurred


Определим ID пользователей, которым выданы книги Толстого, которым также выданы книги Чехова.

In [31]:
customer_id = '''SELECT Customer_ID FROM Bookinuse
AS tols_user
WHERE Author='Толстой'
AND EXISTS (SELECT
Customer_ID FROM Bookinuse
WHERE Author='Чехов'
AND Customer_ID=tols_user.Customer_id)
'''

In [32]:
execute_query(connection, customer_id)

Query executed successfully


In [33]:
customers = execute_read_query(connection, customer_id)

for i in customers:
    print(i)

Определим ID пользователей, которым выданы книги Чехова, и которым при этом не выданы книги Ильфа и Петрова.

In [34]:
customer_id = '''SELECT Customer_ID FROM Bookinuse
    AS cheh_user
    WHERE Author='Чехов'
    AND NOT EXISTS (SELECT
    Customer_ID FROM Bookinuse
    WHERE Author='Ильф и Петров'
    AND Customer_ID=cheh_user.Customer_id)
'''

In [35]:
execute_query(connection, customer_id)

Query executed successfully


In [36]:
customers = execute_read_query(connection, customer_id)

for i in customers:
    print(i)

Определим ID пользователей, которым выданы книги авторов, книги которых выданы пользователю с ID 31. 

In [37]:
customer_id = '''SELECT Customer_ID
      FROM Bookinuse WHERE Author IN (SELECT 
      Author FROM Bookinuse
      WHERE Customer_ID=31);
      '''

In [38]:
execute_query(connection, customer_id)

Query executed successfully


In [39]:
customers = execute_read_query(connection, customer_id)

for i in customers:
    print(i)

Определим ID пользователей, которым выдана хотя бы одна книга Пастернака, и которым при этом выдано более 2 книг.

In [40]:
customer_id = '''SELECT Customer_ID FROM Bookinuse
       AS pas_user
       WHERE EXISTS (SELECT
       Customer_ID FROM Bookinuse
       WHERE Author='Пастернак'
       AND Customer_ID=pas_user.Customer_ID)
       GROUP BY Customer_ID   
       HAVING COUNT(Title) > 2
'''

In [41]:
execute_query(connection, customer_id)

Query executed successfully


In [42]:
customers = execute_read_query(connection, customer_id)

for i in customers:
    print(i)

Определим авторов, книги которых выданы пользователю по фамилии Краснов.

In [43]:
customer_id = '''SELECT DISTINCT Author 
       FROM Bookinuse bk 
       WHERE EXISTS (SELECT * 
       FROM Customer cs 
       WHERE cs.Customer_ID=bk.Customer_ID
       AND Surname='Краснов')
'''

In [44]:
execute_query(connection, customer_id)

Query executed successfully


In [45]:
customers = execute_read_query(connection, customer_id)

for i in customers:
    print(i)

### Создание таблицы в БД из .csv файла

In [46]:
connection = create_connection("C:\\Disc_D\\DS_course_mipt\\ОТ\\Python_DA\\БД_SQL\\customers.db")

Connection to SQLite DB successful


In [47]:
campaign_data = """
CREATE TABLE IF NOT EXISTS CampaignData(
  campaign_id INTEGER, 
  campaign_type TEXT NOT NULL, 
  start_date TEXT NOT NULL, 
  end_date INTEGER NOT NULL
);
"""

In [48]:
execute_query(connection, campaign_data)

Query executed successfully


In [49]:
campaign_data = pd.read_csv('./data/campaign_data.csv')
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [50]:
campaign_data.to_sql('CampaignData', connection, if_exists='append', index=False)

28

In [51]:
customer_id = '''SELECT * 
       FROM CampaignData;
'''
customers = execute_read_query(connection, customer_id)

for i in customers:
    print(i)

(24, 'Y', '21/10/13', '20/12/13')
(25, 'Y', '21/10/13', '22/11/13')
(20, 'Y', '07/09/13', '16/11/13')
(23, 'Y', '08/10/13', '15/11/13')
(21, 'Y', '16/09/13', '18/10/13')
(22, 'X', '16/09/13', '18/10/13')
(18, 'X', '10/08/13', '04/10/13')
(19, 'Y', '26/08/13', '27/09/13')
(17, 'Y', '29/07/13', '30/08/13')
(16, 'Y', '15/07/13', '16/08/13')
(13, 'X', '19/05/13', '05/07/13')
(11, 'Y', '22/04/13', '07/06/13')
(12, 'Y', '22/04/13', '24/05/13')
(10, 'Y', '08/04/13', '10/05/13')
(9, 'Y', '11/03/13', '12/04/13')
(8, 'X', '16/02/13', '05/04/13')
(7, 'Y', '02/02/13', '08/03/13')
(6, 'Y', '28/01/13', '01/03/13')
(3, 'Y', '22/12/12', '16/02/13')
(5, 'Y', '12/01/13', '15/02/13')
(4, 'Y', '07/01/13', '08/02/13')
(1, 'Y', '12/12/12', '18/01/13')
(2, 'Y', '17/12/12', '18/01/13')
(30, 'X', '19/11/12', '04/01/13')
(29, 'Y', '08/10/12', '30/11/12')
(28, 'Y', '16/09/12', '16/11/12')
(27, 'Y', '25/08/12', '27/10/12')
(26, 'X', '12/08/12', '21/09/12')
(24, 'Y', '21/10/13', '20/12/13')
(25, 'Y', '21/10/13', '

In [52]:
execute_query(connection, customer_id)

Query executed successfully


In [53]:
coupon_item_mapping = pd.read_csv('./data/coupon_item_mapping.csv')
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [54]:
coupon_item_mapping.to_sql('coupon_item_mapping', connection, if_exists='append', index=False)

92663

In [55]:
coupon_item_mapping = '''SELECT COUNT(coupon_id)
       FROM coupon_item_mapping
       WHERE item_id <= 30;
        '''

coupon_item_mapping = execute_read_query(connection, coupon_item_mapping)

for i in coupon_item_mapping:
    print(i)

(160,)


In [56]:
customer_demographics = pd.read_csv('./data/customer_demographics.csv')
customer_demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [57]:
customer_demographics.to_sql('customer_demographics', connection, if_exists='append', index=False)

760

In [58]:
customer_demographics = '''SELECT COUNT(DISTINCT customer_id)
       FROM customer_demographics
       WHERE age_range like '7%';
        '''

customer_demographics = execute_read_query(connection, customer_demographics)

for i in customer_demographics:
    print(i)

(68,)


In [59]:
customer_transaction_data = pd.read_csv('./data/customer_transaction_data.csv')
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [60]:
customer_transaction_data.to_sql('customer_transaction_data', connection, if_exists='append', index=False)

1324566

In [61]:
customer_transaction_data = '''SELECT COUNT(customer_id)
       FROM customer_transaction_data
       WHERE quantity !=1;
        '''

customer_transaction_data = execute_read_query(connection, customer_transaction_data)

for i in customer_transaction_data:
    print(i)

(811521,)


In [ ]:
customer_transaction_data = '''SELECT date, c_t.customer_id, c_d.customer_id, item_id, quantity, selling_price, coupon_discount, age_range
       FROM customer_transaction_data as c_t
       LEFT JOIN customer_demographics as c_d
        ON c_t.customer_id = c_d.customer_id;
        '''

customer_transaction_data = execute_read_query(connection, customer_transaction_data)

for i in customer_transaction_data:
    print(i)